In [5]:
import requests as r
from bs4 import BeautifulSoup as bs
import json
from pprint import pp
import pandas as pd

url = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"
response = r.get(url)

pp(response)

<Response [200]>


In [6]:
data = response.json()


In [10]:
# Create a list of dictionaries from the data

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)




In [11]:
# Calculate the percentage of approved applications for self-employed applicants
self_employed_approved = df.loc[(df["self_employed"] == "Yes") & (df["loan_status"] == "Approved")].shape[0]
self_employed_total = df.loc[df["self_employed"] == "Yes"].shape[0]
self_employed_approval_rate = self_employed_approved / self_employed_total * 100

# Plot the approval rate
fig, ax = plt.subplots()
ax.bar(["Self-employed"], [self_employed_approval_rate])
ax.set_ylabel("Approval rate (%)")
ax.set_title("Percentage of Applications Approved for Self-employed Applicants")
plt.show()


KeyError: 'self_employed'

In [ ]:
def execute_query(query):
    df = spark.read.format("jdbc")\
        .option("url", "jdbc:mysql://{}:{}/{}".format(mysql_hostname, mysql_port, mysql_database))\
        .option("driver", "/Users/roy/Downloads/mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar")\
        .option("dbtable", "({})".format(query))\
        .option("user", mysql_username)\
        .option("password", mysql_password)\
        .load()
    return df

def execute_query(query):
    try:
        result = spark.sql(query).toPandas()
        return result
    except Exception as e:
        print("Error executing query: {}".format(e))
        return None

# define a function to validate user input
def validate_input(prompt, expected_type):
    while True:
        user_input = pyin.inputStr(prompt)
        try:
            validated_input = expected_type(user_input)
            return validated_input
        except ValueError:
            print("Invalid input. Please enter a {}.".format(expected_type.__name__))

# define a loop for user interaction
while True:
    print("Select an option:")
    print("1. Display transactions by zip code and month/year")
    print("2. Display number and total value of transactions by type")
    print("3. Display total number and value of transactions by state")
    print("4. Exit")

    # validate user input
    choice = validate_input("Enter your choice: ", int)
    if choice == 1:
        #1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.
       
        zip_code = validate_input("Enter the zip code: ", int)
        year = validate_input("Enter the year (YYYY): ", int)
        month = validate_input("Enter the month (MM): ", int)
        result = execute_query("SELECT * FROM CDW_SAPP_CUSTOMER, CDW_SAPP_BRANCH WHERE CUST_ZIP = {} AND YEAR(date) = {} AND MONTH(date) = {} ORDER BY date DESC".format(zip_code, year, month))
        if result:
            pd_result = result.toPandas()
            print(pd_result)
        else:
            print("No transactions found.")
    elif choice == 2:
        #2)    Used to display the number and total values of transactions for a given type.
      
        transaction_type = validate_input("Enter the transaction type: ", str)
        result = execute_query("SELECT TRANSACTION_VALUE, TRANSACTION_TYPE FROM '{}' where TRANSACTION_TYPE  = '{}'".format(query, transaction_type))
        if result:
            pd_result = result.toPandas()
            print(pd_result)
        else:
            print("No transactions found.")
    elif choice == 3:
        #3)    Used to display the total number and total values of transactions for branches in a given state.
    
        state = validate_input("Enter the state: ", str)
        result = execute_query("SELECT bc.state, COUNT(*) as count, SUM(TRANSACTION_VALUE) as total_value FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE = cc.BRANCH_CODE WHERE BRANCH_STATE = '{}'".format(state))
        if result:
            pd_result = result.toPandas()
            print(pd_result)
        else:
            print("No transactions found.")
    elif choice == 4:
        # exit
        break
    else:
        print("Invalid choice.")
